In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
from sklearn.preprocessing import power_transform
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import pickle

# Data Loading

Source:
    https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather

In [3]:
weather_pred=pd.read_csv('weather_features.csv')
weather_pred

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178391,2018-12-31 19:00:00+01:00,Seville,287.760,287.150,288.150,1028,54,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178392,2018-12-31 20:00:00+01:00,Seville,285.760,285.150,286.150,1029,62,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178393,2018-12-31 21:00:00+01:00,Seville,285.150,285.150,285.150,1028,58,4,50,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178394,2018-12-31 22:00:00+01:00,Seville,284.150,284.150,284.150,1029,57,4,60,0.0,0.0,0.0,0,800,clear,sky is clear,01n


        Sample of above dataset

In [4]:
# take sample of above dataset
weather_pred=weather_pred.sample(n=1000,random_state=42)
weather_pred

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
61043,2017-11-23 19:00:00+01:00,Madrid,286.730000,284.150000,288.150000,1020,41,0,220,0.0,0.0,0.0,0,800,clear,sky is clear,01n
96079,2017-10-02 14:00:00+02:00,Bilbao,292.640000,292.150000,293.150000,1026,94,0,317,0.0,0.0,0.0,75,701,mist,mist,50d
92849,2017-05-25 08:00:00+02:00,Bilbao,287.750000,284.150000,292.150000,1014,82,0,0,0.0,0.0,0.0,0,800,clear,sky is clear,01d
163112,2017-04-14 03:00:00+02:00,Seville,291.860000,285.150000,301.150000,1020,93,1,0,0.0,0.0,0.0,32,802,clouds,scattered clouds,03n
159577,2016-11-22 11:00:00+01:00,Seville,291.990000,285.150000,303.150000,1005,87,0,0,0.0,0.0,0.0,20,741,fog,fog,50d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,2015-03-28 21:00:00+01:00,Valencia,289.041656,289.041656,289.041656,1023,49,1,16,0.0,0.0,0.0,0,800,clear,sky is clear,01
54770,2017-03-11 22:00:00+01:00,Madrid,289.150000,289.150000,289.150000,1018,55,3,300,0.0,0.0,0.0,0,800,clear,sky is clear,01n
86313,2016-09-04 14:00:00+02:00,Bilbao,304.820000,300.150000,311.480000,1021,69,3,290,0.0,0.0,0.0,20,801,clouds,few clouds,02d
154369,2016-04-20 14:00:00+02:00,Seville,292.890000,286.150000,303.150000,1016,72,5,210,0.3,0.0,0.0,75,211,thunderstorm,proximity thunderstorm,11d


# DataPreprocessing 

    1. Extracting date/time/month/year/hour/day/weekday/quarter

In [5]:
weather_pred['date']=pd.to_datetime(weather_pred['dt_iso'].str[:10])
weather_pred['time']=pd.to_datetime(weather_pred['dt_iso'].str[11:19])
weather_pred['month']=weather_pred['date'].dt.month
weather_pred['year']=weather_pred['date'].dt.year
weather_pred['hour']=weather_pred['time'].dt.hour
weather_pred['day']=weather_pred['date'].dt.day
weather_pred['weekday']=weather_pred['date'].dt.weekday
weather_pred['quarter']=weather_pred['date'].dt.quarter

In [6]:
weather_pred

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,...,weather_description,weather_icon,date,time,month,year,hour,day,weekday,quarter
61043,2017-11-23 19:00:00+01:00,Madrid,286.730000,284.150000,288.150000,1020,41,0,220,0.0,...,sky is clear,01n,2017-11-23,2024-03-25 19:00:00,11,2017,19,23,3,4
96079,2017-10-02 14:00:00+02:00,Bilbao,292.640000,292.150000,293.150000,1026,94,0,317,0.0,...,mist,50d,2017-10-02,2024-03-25 14:00:00,10,2017,14,2,0,4
92849,2017-05-25 08:00:00+02:00,Bilbao,287.750000,284.150000,292.150000,1014,82,0,0,0.0,...,sky is clear,01d,2017-05-25,2024-03-25 08:00:00,5,2017,8,25,3,2
163112,2017-04-14 03:00:00+02:00,Seville,291.860000,285.150000,301.150000,1020,93,1,0,0.0,...,scattered clouds,03n,2017-04-14,2024-03-25 03:00:00,4,2017,3,14,4,2
159577,2016-11-22 11:00:00+01:00,Seville,291.990000,285.150000,303.150000,1005,87,0,0,0.0,...,fog,50d,2016-11-22,2024-03-25 11:00:00,11,2016,11,22,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,2015-03-28 21:00:00+01:00,Valencia,289.041656,289.041656,289.041656,1023,49,1,16,0.0,...,sky is clear,01,2015-03-28,2024-03-25 21:00:00,3,2015,21,28,5,1
54770,2017-03-11 22:00:00+01:00,Madrid,289.150000,289.150000,289.150000,1018,55,3,300,0.0,...,sky is clear,01n,2017-03-11,2024-03-25 22:00:00,3,2017,22,11,5,1
86313,2016-09-04 14:00:00+02:00,Bilbao,304.820000,300.150000,311.480000,1021,69,3,290,0.0,...,few clouds,02d,2016-09-04,2024-03-25 14:00:00,9,2016,14,4,6,3
154369,2016-04-20 14:00:00+02:00,Seville,292.890000,286.150000,303.150000,1016,72,5,210,0.3,...,proximity thunderstorm,11d,2016-04-20,2024-03-25 14:00:00,4,2016,14,20,2,2


In [7]:
# Prepare the data for analysis.
# Drop the columns that are not required for analysis.
weather_pred.drop(['dt_iso','city_name','weather_description','date','time'],axis=1,inplace=True)

In [8]:
weather_pred

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
61043,286.730000,284.150000,288.150000,1020,41,0,220,0.0,0.0,0.0,0,800,clear,01n,11,2017,19,23,3,4
96079,292.640000,292.150000,293.150000,1026,94,0,317,0.0,0.0,0.0,75,701,mist,50d,10,2017,14,2,0,4
92849,287.750000,284.150000,292.150000,1014,82,0,0,0.0,0.0,0.0,0,800,clear,01d,5,2017,8,25,3,2
163112,291.860000,285.150000,301.150000,1020,93,1,0,0.0,0.0,0.0,32,802,clouds,03n,4,2017,3,14,4,2
159577,291.990000,285.150000,303.150000,1005,87,0,0,0.0,0.0,0.0,20,741,fog,50d,11,2016,11,22,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,289.041656,289.041656,289.041656,1023,49,1,16,0.0,0.0,0.0,0,800,clear,01,3,2015,21,28,5,1
54770,289.150000,289.150000,289.150000,1018,55,3,300,0.0,0.0,0.0,0,800,clear,01n,3,2017,22,11,5,1
86313,304.820000,300.150000,311.480000,1021,69,3,290,0.0,0.0,0.0,20,801,clouds,02d,9,2016,14,4,6,3
154369,292.890000,286.150000,303.150000,1016,72,5,210,0.3,0.0,0.0,75,211,thunderstorm,11d,4,2016,14,20,2,2


    4. Encoding the columns

In [9]:
# label encoding
le=LabelEncoder()
weather_pred['weather_main']=le.fit_transform(weather_pred['weather_main'])
weather_pred['weather_icon']=le.fit_transform(weather_pred['weather_icon'])
weather_pred

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
61043,286.730000,284.150000,288.150000,1020,41,0,220,0.0,0.0,0.0,0,800,0,2,11,2017,19,23,3,4
96079,292.640000,292.150000,293.150000,1026,94,0,317,0.0,0.0,0.0,75,701,6,21,10,2017,14,2,0,4
92849,287.750000,284.150000,292.150000,1014,82,0,0,0.0,0.0,0.0,0,800,0,1,5,2017,8,25,3,2
163112,291.860000,285.150000,301.150000,1020,93,1,0,0.0,0.0,0.0,32,802,1,8,4,2017,3,14,4,2
159577,291.990000,285.150000,303.150000,1005,87,0,0,0.0,0.0,0.0,20,741,4,21,11,2016,11,22,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,289.041656,289.041656,289.041656,1023,49,1,16,0.0,0.0,0.0,0,800,0,0,3,2015,21,28,5,1
54770,289.150000,289.150000,289.150000,1018,55,3,300,0.0,0.0,0.0,0,800,0,2,3,2017,22,11,5,1
86313,304.820000,300.150000,311.480000,1021,69,3,290,0.0,0.0,0.0,20,801,1,4,9,2016,14,4,6,3
154369,292.890000,286.150000,303.150000,1016,72,5,210,0.3,0.0,0.0,75,211,9,17,4,2016,14,20,2,2


In [10]:
# Remove the outliers.
z=np.abs(zscore(weather_pred))
z

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
61043,0.369296,0.541571,0.344783,0.030272,1.310089,1.213242,0.437668,0.254304,0.076458,0.044492,0.823703,0.388908,0.620394,0.693346,1.311155,0.351944,1.057556,0.873237,0.015058,1.344689
96079,0.370441,0.464322,0.242546,0.028353,1.159228,1.213242,1.264437,0.254304,0.076458,0.044492,1.567261,0.547081,1.943218,2.571481,1.019722,0.351944,0.327905,1.500304,1.520886,1.344689
92849,0.241626,0.541571,0.125080,0.032192,0.600137,1.213242,1.437479,0.254304,0.076458,0.044492,0.823703,0.388908,0.620394,0.865179,0.437440,0.351944,0.547676,1.099289,0.015058,0.447035
163112,0.272811,0.415834,1.182273,0.030272,1.112637,0.714581,1.437479,0.254304,0.076458,0.044492,0.196442,0.407817,0.193125,0.337652,0.728873,0.351944,1.277327,0.143995,0.486884,0.447035
159577,0.289082,0.415834,1.417205,0.035071,0.833091,1.213242,1.437479,0.254304,0.076458,0.044492,0.186113,0.168904,1.088681,2.571481,1.311155,0.525722,0.109885,0.760212,1.018943,1.344689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2086,0.079953,0.073489,0.240044,0.029313,0.937362,0.714581,1.301105,0.254304,0.076458,0.044492,0.823703,0.388908,0.620394,1.037012,1.020305,1.403388,1.349417,1.438366,0.988827,1.342897
54770,0.066392,0.087112,0.227317,0.030912,0.657817,0.282741,1.119539,0.254304,0.076458,0.044492,0.823703,0.388908,0.620394,0.693346,1.020305,0.351944,1.495347,0.483072,0.988827,1.342897
86313,1.894975,1.470214,2.395696,0.029952,0.005544,0.282741,1.034305,0.254304,0.076458,0.044492,0.186113,0.398363,0.193125,0.349680,0.728290,0.525722,0.327905,1.274253,1.490769,0.448827
154369,0.401733,0.290098,1.417205,0.031552,0.134228,1.280063,0.352434,0.928507,0.076458,0.044492,1.567261,5.179755,3.225024,1.884149,0.728873,0.525722,0.327905,0.534160,0.517001,0.447035


In [11]:
threshold=3
weather_pred_new=weather_pred[(z<3).all(axis=1)]
weather_pred_new

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
61043,286.730000,284.150000,288.150000,1020,41,0,220,0.0,0.0,0.0,0,800,0,2,11,2017,19,23,3,4
96079,292.640000,292.150000,293.150000,1026,94,0,317,0.0,0.0,0.0,75,701,6,21,10,2017,14,2,0,4
92849,287.750000,284.150000,292.150000,1014,82,0,0,0.0,0.0,0.0,0,800,0,1,5,2017,8,25,3,2
163112,291.860000,285.150000,301.150000,1020,93,1,0,0.0,0.0,0.0,32,802,1,8,4,2017,3,14,4,2
159577,291.990000,285.150000,303.150000,1005,87,0,0,0.0,0.0,0.0,20,741,4,21,11,2016,11,22,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64963,291.160000,290.150000,292.150000,1021,48,2,110,0.0,0.0,0.0,0,800,0,2,4,2018,23,19,3,2
2086,289.041656,289.041656,289.041656,1023,49,1,16,0.0,0.0,0.0,0,800,0,0,3,2015,21,28,5,1
54770,289.150000,289.150000,289.150000,1018,55,3,300,0.0,0.0,0.0,0,800,0,2,3,2017,22,11,5,1
86313,304.820000,300.150000,311.480000,1021,69,3,290,0.0,0.0,0.0,20,801,1,4,9,2016,14,4,6,3


    6. Check for skewness and remove (if any)

In [12]:
weather_pred_new.skew()

temp             0.108515
temp_min         0.071949
temp_max         0.166397
pressure        -2.479382
humidity        -0.534699
wind_speed       0.934703
wind_deg        -0.061927
rain_1h          2.972225
rain_3h         16.786010
snow_3h          0.000000
clouds_all       1.046998
weather_id      -2.828982
weather_main     2.079747
weather_icon     1.380475
month           -0.068807
year            -0.101281
hour            -0.010718
day              0.115696
weekday         -0.032046
quarter         -0.019987
dtype: float64

# Exploratory Data Analysis

    Check for correlation
            Making different graphs for checking relationships between the variables 

In [13]:
# find correlation
weather_pred_new.corr()

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
temp,1.000000,0.976685,0.967936,0.030128,-0.551700,0.192955,-0.002900,-0.152115,-0.046418,NaN,-0.192747,0.180833,-0.217835,-0.247878,0.236477,0.031303,0.225819,0.015698,-0.011011,0.232655
temp_min,0.976685,1.000000,0.897396,0.007106,-0.530468,0.193306,0.013096,-0.134602,-0.036965,NaN,-0.169615,0.166156,-0.210881,-0.252810,0.225044,0.054709,0.226708,0.027735,-0.013010,0.224747
temp_max,0.967936,0.897396,1.000000,0.053766,-0.526526,0.176168,-0.028687,-0.160035,-0.054108,NaN,-0.205365,0.182075,-0.207445,-0.218915,0.231018,0.005815,0.205890,0.002880,-0.009635,0.224851
pressure,0.030128,0.007106,0.053766,1.000000,0.040949,-0.114596,-0.026912,-0.097340,-0.002712,NaN,-0.098629,0.078100,-0.051726,-0.032653,0.108313,0.077078,-0.032290,0.003879,0.011188,0.073207
humidity,-0.551700,-0.530468,-0.526526,0.040949,1.000000,-0.258534,-0.000435,0.238678,0.029105,NaN,0.386579,-0.280778,0.373080,0.453618,0.008955,-0.012178,-0.311837,0.009763,0.023699,0.009885
wind_speed,0.192955,0.193306,0.176168,-0.114596,-0.258534,1.000000,0.247223,0.000018,0.002793,NaN,0.077966,0.035137,-0.018860,-0.013017,-0.126733,0.070806,0.199889,-0.021716,-0.055691,-0.127910
wind_deg,-0.002900,0.013096,-0.028687,-0.026912,-0.000435,0.247223,1.000000,0.057652,0.034871,NaN,0.070383,-0.033866,0.024370,0.018298,-0.059176,-0.005511,0.005277,-0.071586,-0.017159,-0.052586
rain_1h,-0.152115,-0.134602,-0.160035,-0.097340,0.238678,0.000018,0.057652,1.000000,-0.019054,NaN,0.411433,-0.959835,0.836043,0.519639,-0.089303,-0.100696,0.036092,0.005240,-0.013528,-0.088701
rain_3h,-0.046418,-0.036965,-0.054108,-0.002712,0.029105,0.002793,0.034871,-0.019054,1.000000,NaN,0.028348,0.022239,-0.016378,-0.011041,-0.037296,-0.073068,-0.037014,0.002704,0.015671,-0.031942
snow_3h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Feature Selection

    Train/Test Splitting the Data

In [14]:
# columns: temp	temp_min, temp_max, pressure, humidity, wind_speed, wind_deg, rain_1h, rain_3h, snow_3h,clouds_all,	weather_id,weather_main,weather_icon,month,	year,hour,day,weekday,quarter

X= weather_pred_new.drop('temp',axis=1)
y= weather_pred_new['temp']

In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

    Best feature Selection
        Using Random Forest Regressor

In [16]:
k=5
model = RandomForestRegressor()
model.fit(X, y)
importances = model.feature_importances_
selected_feature_indices = np.argsort(importances)[-k:]
selected_features = X.columns[selected_feature_indices]

In [17]:
selected_features

Index(['wind_deg', 'humidity', 'pressure', 'temp_max', 'temp_min'], dtype='object')

# Model Selection and Training

    Apply Linear Regression

In [18]:
# depending on above selected features apply linear regression
X_train = X_train[selected_features]
X_test = X_test[selected_features]

In [19]:
X_train

,wind_deg,humidity,pressure,temp_max,temp_min
134830,0,69,1017,277.150,277.150
116237,320,75,1015,283.150,275.370
177509,190,81,1018,284.150,283.150
107830,247,59,1005,282.970,282.970
123390,290,100,1023,293.389,293.389
...,...,...,...,...,...
131946,290,100,1024,291.150,289.150
66711,20,73,1010,294.150,291.150
99791,30,65,1014,281.150,280.150
86389,340,65,1012,310.930,299.150


In [20]:
X_test

,wind_deg,humidity,pressure,temp_max,temp_min
31047,70,83,1016,299.15,299.15
173075,220,68,1015,292.15,290.15
4019,0,60,1020,294.35,294.15
63494,0,47,1006,277.15,269.15
64445,240,65,1007,280.15,278.15
...,...,...,...,...,...
170605,270,71,1022,287.15,286.15
92849,0,82,1014,292.15,284.15
94570,0,58,1018,303.15,301.15
165965,120,77,1016,290.15,289.15


    Apply Decision Tree

In [21]:
# apply decision tree
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
y_pred_dt



array([299.15      , 291.15      , 294.481     , 274.15      ,
       279.19      , 289.34      , 284.65      , 281.58      ,
       283.23      , 292.15      , 293.15      , 274.15      ,
       283.94      , 288.39      , 288.084     , 298.15      ,
       286.7475    , 280.58      , 289.15      , 292.04      ,
       274.15      , 305.12      , 305.12      , 305.15      ,
       289.1       , 292.25      , 287.94      , 292.15      ,
       291.88      , 296.539     , 284.92      , 286.12      ,
       300.65      , 302.15      , 298.65      , 295.15      ,
       290.68      , 273.15      , 305.15      , 285.15      ,
       294.56      , 286.15      , 289.65      , 306.57      ,
       295.01      , 291.25      , 295.6       , 289.05      ,
       300.15      , 298.65      , 283.85      , 287.15      ,
       287.4315    , 286.15      , 283.042     , 288.74      ,
       278.789     , 291.15      , 280.65      , 289.54      ,
       287.14      , 288.54      , 288.44      , 285.84

    Model Evaluation: Decision Tree

In [22]:
# CALCULATE ACCURACY
print('Accuracy score:',dt.score(X_test,y_test))
print('R2 score:',r2_score(y_test,y_pred_dt))
print('Mean Absolute Error:',mean_absolute_error(y_test,y_pred_dt))
print('Mean Squared Error:',mean_squared_error(y_test,y_pred_dt))
print('Root Mean Squared Error:',np.sqrt(mean_squared_error(y_test,y_pred_dt)))

Accuracy score: 0.9903710667417001
R2 score: 0.9903710667417001
Mean Absolute Error: 0.4046155252659634
Mean Squared Error: 0.6158174875799926
Root Mean Squared Error: 0.7847403950224511


In [24]:
print('Accuracy score:',dt.score(X_test,y_test))

Accuracy score: 0.9903710667417001


# Optimizing the above models by finding the best hyper parameters

    Decision Tree (Optimised)

In [25]:
dt_op = DecisionTreeRegressor()

# Define the hyperparameter grid
parameters = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'criterion': ['squared_error', 'friedman_mse', 'poisson']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=dt_op, param_grid=parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Use the best model to make predictions
y_pred_dt_op = best_model.predict(X_test)


In [ ]:
# applying  decision tree with best parameters
dt_op = DecisionTreeRegressor(criterion='squared_error', max_depth=9, min_samples_leaf=1, min_samples_split=2)
dt_op.fit(X_train, y_train)
y_pred_dt_op = dt_op.predict(X_test)
y_pred_dt_op

array([299.15314   , 290.952595  , 294.3018    , 274.43210844,
       279.25380156, 289.5143    , 284.64895063, 281.6566    ,
       283.226305  , 292.15024   , 293.14964   , 272.69628719,
       284.090095  , 287.2803    , 288.19179   , 298.17236   ,
       286.6260025 , 280.176285  , 289.14772   , 292.221705  ,
       272.14497406, 304.2625    , 304.4063    , 305.07114   ,
       289.0094    , 292.29407781, 287.93615   , 292.095315  ,
       291.56120063, 296.52448   , 284.89489625, 285.8196    ,
       300.375     , 302.612     , 298.59378   , 295.15783   ,
       289.4741    , 271.55114063, 304.7769    , 285.16176   ,
       294.5432    , 286.12242   , 289.46305   , 306.6589    ,
       295.0942    , 290.3468    , 295.59907   , 289.230565  ,
       300.12596688, 298.40136   , 282.4974    , 287.15      ,
       287.377965  , 286.15811   , 283.03502   , 288.69966313,
       279.12845156, 290.981     , 280.03185   , 289.43099625,
       286.497275  , 288.58512   , 288.4711    , 285.43

In [ ]:

# CALCULATE ACCURACY
print('Accuracy score:',dt_op.score(X_test,y_test))
print('R2 score:',r2_score(y_test,y_pred_dt_op))
print('Mean Absolute Error:',mean_absolute_error(y_test,y_pred_dt_op))
print('Mean Squared Error:',mean_squared_error(y_test,y_pred_dt_op))
print('Root Mean Squared Error:',np.sqrt(mean_squared_error(y_test,y_pred_dt_op)))


Accuracy score: 0.9955492500021556
R2 score: 0.9955492500021556
Mean Absolute Error: 0.24676850066492823
Mean Squared Error: 0.28464728210226603
Root Mean Squared Error: 0.5335234597487406


# Creating Pickle files

In [ ]:

# Creating pickle files for Decision Tree Regressor (optimised)
filename_dt = 'weather_pred_dt.pkl'
pickle.dump(dt_op, open(filename_dt, 'wb'))


In [ ]:
# load the model
loaded_model_dt = pickle.load(open(filename_dt, 'rb'))
result_dt = loaded_model_dt.score(X_test, y_test)
print(result_dt)

0.9955492500021556
